In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import datasets, transforms
import torchvision
import matplotlib.pyplot as plt
from torchsummary import summary
import gc


In [ ]:
!nvidia-smi
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Hyper Parameters
EPOCH = 10
BATCH_SIZE = 64
LR = 0.01
MOMENTUM = 0.9
num_classes = 10
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
DOWNLOAD = True

In [ ]:
# Transform
transform = transforms.Compose(
                [
                 transforms.Resize(size=(224,224)),
                 transforms.ToTensor(),
                 transforms.Normalize((0.5,), (0.5,)),                 
                #  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                 transforms.RandomHorizontalFlip(),
                ]
                )

# Data
train_dataset = datasets.CIFAR10(root='/dataset', train=True, download=DOWNLOAD, transform=transform)
valid_dataset = datasets.CIFAR10(root='/dataset', train=False, download=DOWNLOAD, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [ ]:
class Vgg16(nn.Module):
    def __init__(self, num_classes):
        super(Vgg16, self).__init__()
        #block 1
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        #block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )
        #block 3
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )
        #block 4
        self.block4 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
        )
        #block 5
        self.block5 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding='same'),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
        )
        self.MaxPool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.classfier = nn.Sequential(
            nn.Linear(in_features=7*7*512, out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.4),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.4),
            nn.Linear(in_features=4096, out_features=num_classes),            
        )
                        
    def forward(self, x):          
          x = self.block1(x)          
          x = self.MaxPool2d(x)          
          x = self.block2(x)          
          x = self.MaxPool2d(x)          
          x = self.block3(x)          
          x = self.MaxPool2d(x)          
          x = self.block4(x)          
          x = self.MaxPool2d(x)          
          x = self.block5(x)          
          x = self.MaxPool2d(x)                
          x = torch.flatten(x,1)
          x = self.classfier(x)
        
          return x

In [ ]:
model = Vgg16(num_classes)
model = model.to(device)
print(model)
summary(model, (3,224,224))
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM)
criterion = nn.CrossEntropyLoss()



In [ ]:
def train(epoch):
    print('\nEpoch: %d' % epoch)    
    model.train()
    
    train_loss = 0
    correct = 0
    total = 0            
    for batch_idx, (inputs, targets) in enumerate(train_loader):        
        inputs, targets = inputs.to(device), targets.to(device)
                
        optimizer.zero_grad()        
        outputs = model(inputs)
                
        loss = criterion(outputs, targets)                
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 100 == 0:
            train_acc_per_epoch = 100 * (correct / float(total))
            train_loss_per_epoch = train_loss / total
            print("Train Epoch: {}/{} [iter： {:03d}/{:03d}], acc： {:.6f}, loss： {:.6f}".format(
               epoch, EPOCH, batch_idx+1, len(train_loader),
               train_acc_per_epoch,
               train_loss_per_epoch))
    return train_acc_per_epoch, train_loss_per_epoch


In [ ]:
def test(epoch):        
    model.eval()
    test_loss = 0
    correct = 0
    total = 0    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(valid_loader):            
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs)            
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            if batch_idx % 100 == 0:     
              test_acc_per_epoch = 100 * (correct / float(total))
              test_loss_per_epoch = test_loss / total
              print("Valid Epoch: {}/{} [iter： {:03d}/{:03d}], acc： {:.6f}, loss： {:.6f}".format(
                epoch, EPOCH, batch_idx+1, len(valid_loader),                
                test_acc_per_epoch,
                test_loss_per_epoch))                
            
    return test_acc_per_epoch, test_loss_per_epoch

In [ ]:
def training_loop():
    # set objects for storing metrics
    total_train_loss = []
    total_valid_loss = []
    total_train_accuracy = []
    total_valid_accuracy = []
 
    # Train model
    for epoch in range(1,EPOCH+1):
        # training
        
        train_acc_, train_loss_ = train(epoch)
        total_train_accuracy.append(train_acc_)
        total_train_loss.append(train_loss_)
        
        valid_acc_, valid_loss_ = test(epoch)        
        total_valid_accuracy.append(valid_acc_)
        total_valid_loss.append(valid_loss_)

        print('==========================================================================')
        print("Epoch: {:05d}/{:05d}， Train acc： {:.6f}， Train loss： {:.6f}， Valid acc： {:.6f}， Valid loss： {:.6f}".format(
               epoch, EPOCH, 
               train_acc_, train_loss_,
               valid_acc_, valid_loss_))
        print('==========================================================================')

    # print("====== END ==========")

    return total_train_loss, total_valid_loss, total_train_accuracy, total_valid_accuracy

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
total_train_loss, total_valid_loss, total_train_accuracy, total_valid_accuracy = training_loop()

In [ ]:
def plot_result(total_train, total_valid, label):
    plt.plot(range(1,EPOCH+1), total_train, 'b-', label=f'Training_{label}')
    plt.plot(range(1,EPOCH+1), total_valid, 'g-', label=f'validation_{label}')
    plt.title(f'Training & Validation {label}')
    plt.xlabel('Number of epochs')
    plt.ylabel(f'{label}')
    plt.legend()
    plt.show()

In [ ]:
plot_result(total_train_accuracy, total_valid_accuracy, 'accuracy')
plot_result(total_train_loss, total_valid_loss, 'loss')